In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
if os.getcwd().endswith("experiments"):
    os.chdir("..")

os.getcwd()

'/home/lars/code/python/context-aware-flow-matching'

In [3]:
from src.models import Model
from src.datasets import ModelNet10Dataset

from torch.utils.data import random_split
from lightning import seed_everything

In [4]:
seed_everything(42)

Seed set to 42


42

In [5]:
dataset = ModelNet10Dataset("data")

Found non-empty ModelNet10Dataset in data, skipping download...


preloading meshes: 100%|██████████| 4899/4899 [00:44<00:00, 111.04it/s]


In [6]:
train_data, val_data, test_data = random_split(dataset, [0.8, 0.1, 0.1])

In [7]:
hparams = dict(
    batch_size=24,
    accumulate_batches=16,
    max_epochs=1000,
    gamma=0.5,
    lr_scheduler=dict(
        name="OneCycleLR",
        kwargs=dict(
            max_lr=3e-3,
            div_factor=3,
            final_div_factor=1e4,
        )
    ),
    gradient_clip=0.1,
    track_grad_norm=None,
    num_workers=10,
    accelerator="gpu",
    devices=-1,
)

In [8]:
model = Model(hparams, train_data, val_data, test_data)

In [ ]:
model.fit(trainer_kwargs=dict(log_every_n_steps=1))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 26.8 M
1 | flow    | Sequential | 19.5 M
---------------------------------------
46.4 M    Trainable params
0         Non-trainable params
46.4 M    Total params
185.439   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/lars/code/python/context-aware-flow-matching/src/lightning-trainable/src/lightning_trainable/metrics/sinkhorn.py:71: Sinkhorn did not converge. Consider increasing epsilon or number of iterations.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]